In [ ]:
#Requerimientos necesarios
!pip install -U spacy==3.2.4
!python -m spacy download es_core_news_md
!pip install py2neo==2021.2.3
!pip install neo4j==4.2.0
import numpy as np
import pandas as pd
import spacy
import es_core_news_md
import time
from py2neo import Graph, Node, Relationship

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.4 MB 58.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def crear_grafo_de_conocimiento(data_frame,riesgo_de_interes,informacion_base_remota,usar_modelo_spacy=True):
  """
  data_frame  (str): recibe direccion o path de de los datos a utilizar, 
                     idealmente se esta pensando en un archivo xlsx (excel) 
                     El data frame debe contener las columnas 'Nota' que contiene las noticias
                     y 'Riesgo' que contiene el tipo de Riesgo de las noticias

  riesgo_de_interes: Si se desea crear el grafo de interes a un tipo de riesgo 
                     en especifico por ejemplo 'Ambiental','Security'
                     Si se desea crear para todas las noticias, tambien se 
                     puede especificar 'None'

  informacion_base_remota (str1,str2,str3): Tupla de 3 elementos que contiene 
                                           los datos necesarios  para conectarse 
                                           remotamente a la base de datos de Neo4j
                                           str1: url  
                                           str2: user , user usualmente es neo4j
                                           str3: password 

  usar_modelo_spacy :  True o False, por defecto True, utiliza el modelo spacy 
                      preentrenado para identificar las entidades en los textos 
  """

  #Establecer conexion con la base de datos
  graph = Graph(informacion_base_remota[0],auth=(informacion_base_remota[1],informacion_base_remota[2]))
  graph.delete_all()
  graph.run("CALL apoc.schema.assert({},{},true) YIELD label, key RETURN *")
  #Comenzar con la lectura del archivo
  #Eliminar noticias que pudieran estar dañadas
  df=pd.read_excel(data_frame)  
  df=df[df['Nota'].notnull()]             
  df=df[df['Riesgo'].notnull()]           
  ##Filtrando df por tipo de riesgo
  if riesgo_de_interes !=None:
    df_riesgo = df[df['Riesgo']==riesgo_de_interes]

  df_notas = df_riesgo['Nota']
  if usar_modelo_spacy: 
    ##Carga de modelo
    import es_core_news_md
    nlp = spacy.load("es_core_news_md")
    entidades= ['PER','LOC','ORG','MISC'] 
    ##Notas: Estas entidades estan asociadas al modelo del modelo por defecto en Spacy
  else:
    nlp = spacy.load("/content/drive/MyDrive/modelos_estancia/model-best", disable = ['parser', 'tagger'])
    nlp.add_pipe('sentencizer')
    entidades= ['PERSONAS','LOCACIONES','ORGANIZACIONES','OTROS','PUESTOSDETRABAJO'] 
  ##Notas: Estas entidades estan asociadas al modelo creado por nosotros
  #filtro para elegir, las entidades de interes
  #########################################
  tipo_entidades = [] 
  entities = []   
                                ##Numero de noticias  a procesar
  for article in df_notas[::-1][:200]:
      tokens = nlp(''.join(article))
      gpe_list = []
      ent_list = []
      for ent in tokens.ents:
        if ent.label_ in entidades:
            gpe_list.append(ent.text)
            ent_list.append(ent.label_)
      entities.append(gpe_list)
      tipo_entidades.append(ent_list)    
  articulos = []
      #Limpiar tokens con palabras mal identificadas, stopwords y otros tokens
      #que no son de nuestro interes
  for entity_list in entities:
      cleaned_entity_list = []
      for entity in entity_list:
        a=(entity.lstrip('Los').replace("'s", "").replace("’s",""))
        b=a.replace("Respecto", "").replace("¡Basta", "").replace("“", "")
        c=b.lstrip('').replace('Comentó',"")
        cleaned_entity_list.append(c)
      articulos.append(cleaned_entity_list)
      ## Despues de ello, podrian quedarnos algunas noticias vacias
      ## Filtrar noticias vacias
  articulos_tipo_entidades = [tipo_entidades[i] for i,article in enumerate(articulos) if article != []]
  articulos = [article for article in articulos if article != []]

  articulos_stopwords = []
  articulos_tipo_entidades_stopwords = []
      ##No solo nos pudieran quedar noticias vacias, sino tambien noticias que tengan tokens del tipo: ''
      ##estos tokens no aportan informacion por lo que son removidos
  for contador_1 , i in enumerate(articulos):
    noticiai = []
    entidadesi = []
    for contador_2, j in enumerate(i):
      if j !='':
        noticiai.append(j)
        entidadesi.append(articulos_tipo_entidades[contador_1][contador_2])
    articulos_stopwords.append(noticiai)
    articulos_tipo_entidades_stopwords.append(entidadesi)
  graph.delete_all()
  ##Codigo que nos permite 
  ##establecer las relaciones implicitas entre entidades
  for c ,entities in enumerate(articulos_stopwords):   ##para cada noticia
      if len(entities) >3:
          for i in range(len(entities)-1):             ##para cada entidad en la noticia
            a = Node(articulos_tipo_entidades_stopwords[c][i],name=str(entities[i]) ,
                    municipio='zapopan',fecha='11/10/20')              ##crear un nodo 
            b = Node(articulos_tipo_entidades_stopwords[c][i+1],name=str(entities[i+1]),
                    municipio= 'guadalajara',fecha='11/10/20')       ##crear un nodo con ciertos atributos
            ab=Relationship(a,'r-implicita',b)                        ##establecer la relacion entre los nodos
            graph.create(ab)                                                                                          
  """
  parche del dia 03-11-2022
  la función .query() ya no es valida y es sustituida por .run()
  """
  #graph.query("MATCH (n) WITH toLower(n.name) as name, collect(n) as nodes CALL apoc.refactor.mergeNodes(nodes) yield node RETURN *")
  #graph.query("match ()-[r]->()  match (s)-[r]->(e)  with s,e,type(r) as typ, tail(collect(r)) as coll  foreach(x in coll | delete x)")
  graph.run("MATCH (n) WITH toLower(n.name) as name, collect(n) as nodes CALL apoc.refactor.mergeNodes(nodes) yield node RETURN *")
  graph.run("match ()-[r]->()  match (s)-[r]->(e)  with s,e,type(r) as typ, tail(collect(r)) as coll  foreach(x in coll | delete x)")

In [ ]:
"""
#Por ejemplo se podria utilizar los siguientes datos para utilizar la funcion
ruta = '/content/drive/MyDrive/estancia_preprofesional/CIMAT_ESTANCIA.xlsx'
url = "neo4j+s://145533a5.databases.neo4j.io"
user = "neo4j"
password= "aF07ZbB2Mbbh1ncWRK0Uz3lmICtIv9bMw8VtKzVd5cw"
informacion = [url,user,password] 
interes='Security'
#Utilizar funcion
crear_grafo_de_conocimiento(data_frame=ruta,riesgo_de_interes=interes,informacion_base_remota=informacion,usar_modelo_spacy=True)
"""

'\n#Por ejemplo se podria utilizar los siguientes datos para utilizar la funcion\nruta = \'/content/drive/MyDrive/estancia_preprofesional/CIMAT_ESTANCIA.xlsx\'\nurl = "neo4j+s://145533a5.databases.neo4j.io"\nuser = "neo4j"\npassword= "aF07ZbB2Mbbh1ncWRK0Uz3lmICtIv9bMw8VtKzVd5cw"\ninformacion = [url,user,password] \ninteres=\'Security\'\n#Utilizar funcion\ncrear_grafo_de_conocimiento(data_frame=ruta,riesgo_de_interes=interes,informacion_base_remota=informacion,usar_modelo_spacy=True)\n'

In [ ]:
"""
NEO4J_URI=neo4j+s://73e84c9d.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=ketD21Y-26K6Y2w8Ox-IKjOXoWkJL73a7vjYdZPflB0
AURA_INSTANCENAME=Instance01
"""

'\nNEO4J_URI=neo4j+s://73e84c9d.databases.neo4j.io\nNEO4J_USERNAME=neo4j\nNEO4J_PASSWORD=ketD21Y-26K6Y2w8Ox-IKjOXoWkJL73a7vjYdZPflB0\nAURA_INSTANCENAME=Instance01\n'

In [ ]:
ruta = '/content/drive/MyDrive/estancia_preprofesional/CIMAT_ESTANCIA.xlsx'
url = "bolt://3.239.225.194:7687"
user = "neo4j"
password= "metals-treatments-subordinates"
informacion = [url,user,password] 
interes='Security'
#Utilizar funcion
crear_grafo_de_conocimiento(data_frame=ruta,riesgo_de_interes=interes,informacion_base_remota=informacion,usar_modelo_spacy=True)